해상도를 두배 증가시킨 이미지를 최대한 겹치지않게 random으로 5개 crop하는 코드

In [ ]:
import json
import os
from PIL import Image, ImageDraw
from pycocotools.coco import COCO
from tqdm import tqdm
from datetime import datetime
import random

In [ ]:
def is_overlapping(crop1, crop2, iou_threshold=0.1):
    """
    두 크롭 영역이 겹치는지 IoU(Intersection over Union)로 확인
    """
    x1, y1, w1, h1 = crop1
    x2, y2, w2, h2 = crop2

    # 좌표 계산
    xa = max(x1, x2)
    ya = max(y1, y2)
    xb = min(x1 + w1, x2 + w2)
    yb = min(y1 + h1, y2 + h2)

    # 겹치는 영역의 너비와 높이
    inter_width = max(0, xb - xa)
    inter_height = max(0, yb - ya)
    inter_area = inter_width * inter_height

    # 각 크롭 영역의 넓이
    crop1_area = w1 * h1
    crop2_area = w2 * h2

    # IoU 계산
    iou = inter_area / (crop1_area + crop2_area - inter_area)
    return iou > iou_threshold

def update_annotations_for_subimage(annotations, subimg_info, img_id):
    updated_annotations = []
    x_offset, y_offset, subimg_width, subimg_height = subimg_info

    for ann in annotations:
        x, y, width, height = ann['bbox']

        # BBox가 subimg 영역과 겹치는지 확인
        if (x + width > x_offset and x < x_offset + subimg_width and
            y + height > y_offset and y < y_offset + subimg_height):
            
            # Update BBox coordinate
            new_x = max(x - x_offset, 0)
            new_y = max(y - y_offset, 0)
            width = min(width, x+width - x_offset)
            height = min(height, y+height - y_offset)

            updated_ann = ann.copy()
            updated_ann['bbox'] = [new_x, new_y, width, height]
            updated_ann['image_id'] = img_id
            updated_annotations.append(updated_ann)
    
    return updated_annotations

In [ ]:
def random_crop(image, crop_size, existing_crops):
    img_width, img_height = image.size
    crop_width, crop_height = crop_size
    max_attempts = 100  # 최대 시도 횟수

    for _ in range(max_attempts):
        # 랜덤으로 좌표 선택
        x_offset = random.randint(0, img_width - crop_width)
        y_offset = random.randint(0, img_height - crop_height)
        new_crop = (x_offset, y_offset, crop_width, crop_height)

        # 기존 크롭들과 겹치는지 확인 (IoU 기준)
        if not any(is_overlapping(new_crop, existing_crop) for existing_crop in existing_crops):
            return new_crop

    # 만약 최대 시도 후에도 겹치지 않는 크롭을 찾지 못한 경우, 마지막 크롭 반환
    return new_crop

In [ ]:
def draw_bboxes(image, annotations):
    draw = ImageDraw.Draw(image)
    for ann in annotations:
        x, y, width, height = ann['bbox']
        draw.rectangle([x, y, x+width, y+height], outline="red", width=2)
    return image

In [ ]:
annotation_file = '/data/ephemeral/home/dataset/train.json'

In [ ]:
x2_annotation_path = '/data/ephemeral/home/dataset/train_x2_crop.json'

# Read json
with open(annotation_file, 'r') as file:
    data = json.load(file)

# Modify bbox values
for ann in data['annotations']:
    ann['bbox'][0] *= 2
    ann['bbox'][1] *= 2
    ann['bbox'][2] *= 2
    ann['bbox'][3] *= 2

# Store new json file
with open(x2_annotation_path, 'w') as file:
    json.dump(data, file, indent=2)

In [ ]:
# Read annotation file
annotation_path = '/data/ephemeral/home/dataset/train.json'
dataDir = '/data/ephemeral/home/EDSR-PyTorch/experiment/test/results-Demo/'
subimgs_path = '/data/ephemeral/home/dataset/scale_x2_random'
original_anno_path = '/data/ephemeral/home/dataset/train.json'
updated_annotation_path = '/data/ephemeral/home/dataset/train_random_x2.json'

In [ ]:
# 어노테이션 파일 읽기
with open(annotation_path, 'r') as file:
    data = json.load(file)

# 새로운 이미지와 어노테이션을 위한 리스트 초기화
new_images = []
new_annotations = []
new_img_id = max([img['id'] for img in data['images']]) + 1

# COCO 객체 생성 및 이미지 파일 목록 가져오기
coco = COCO(annotation_path)
image_files = os.listdir(os.path.join(dataDir, 'train'))

crop_size = (1024, 1024)  # 원하는 크롭 크기 설정

for idx in tqdm(image_files, desc="Processing images"):
    img = coco.loadImgs(int(idx.split('_')[0]))[0]
    I = Image.open(os.path.join(dataDir, f"{img['file_name'].split('.')[0]}_x2_SR.png"))
    img_width, img_height = I.size

    # 어노테이션 ID 가져오기
    annIds = coco.getAnnIds(imgIds=img['id'], iscrowd=None)
    anns = coco.loadAnns(annIds)

    existing_crops = []  # 겹치는 크롭을 방지하기 위한 리스트

    # 각 이미지에 대해 5번의 랜덤 크롭 수행
    for i in range(5):
        crop_info = random_crop(I, crop_size, existing_crops)

        updated_anns = update_annotations_for_subimage(anns, crop_info, new_img_id)

        x_offset, y_offset, subimg_width, subimg_height = crop_info
        subimg = I.crop((x_offset, y_offset, x_offset + subimg_width, y_offset + subimg_height))

        subimg_filename = f"{img['file_name'].split('.')[0]}_random_crop_{i}_x2_SR.png"
        
        if updated_anns:
            new_img = {
                "width": subimg_width,
                "height": subimg_height,
                "file_name": subimg_filename,
                "date_captured": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                "id": new_img_id
            }
            new_images.append(new_img)
            new_annotations.extend(updated_anns)

            # bbox가 있는 경우만 subimg 저장 
            subimg.save(os.path.join(subimgs_path, subimg_filename))
        
            new_img_id += 1

        existing_crops.append(crop_info)

print("Updating annotation file...")

In [ ]:
# 원본 train.json 파일에 새로운 데이터 추가
with open(original_anno_path, 'r') as file:
    original_data = json.load(file)

original_data['images'].extend(new_images)
original_data['annotations'].extend(new_annotations)

# 업데이트된 어노테이션 파일 저장
with open(updated_annotation_path, 'w') as file:
    json.dump(original_data, file, indent=2)

print("Annotation file updated successfully.")